In [27]:
import os
import random
import torch
import numpy as np
import torchvision.transforms as transforms
import pydicom
from scipy.ndimage.filters import median_filter
from lungmask import mask
import SimpleITK as sitk
import cv2
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from easyfsl.samplers import TaskSampler
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

from PIL import Image
from torch.utils.data import Dataset
import tensorflow as tf
from torch import nn, optim
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from CustomTaskSampler import TaskSampler

from preprocessing import transform_to_hu,get_mask,preprocess_images
from DICOMDataset import DICOMDataset
from CNN import CNN
from tqdm import tqdm

C:\Users\Nimesha\AppData\Local\Temp\ipykernel_8300\1409341847.py:7: DeprecationWarning: Please use `median_filter` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import median_filter


In [28]:
test_set = DICOMDataset(root_dir='C:/Users/Nimesha/Documents/MSC_RESEARCH/IMAGES/TEST_SET', transform=None)
train_set = DICOMDataset(root_dir='C:/Users/Nimesha/Documents/MSC_RESEARCH/IMAGES/TRAIN_SET/', transform=None)

In [29]:
class PrototypicalNetworks(nn.Module):
    def __init__(self, backbone: nn.Module):
        super(PrototypicalNetworks, self).__init__()
        self.backbone = backbone

    def forward(
        self,
        support_images: torch.Tensor,
        support_labels: torch.Tensor,
        query_images: torch.Tensor,
    ) -> torch.Tensor:
        """
        Predict query labels using labeled support images.
        """
        # Extract the features of support and query images
        z_support = self.backbone.forward(support_images)
        
        z_query = self.backbone.forward(query_images)
        
        
        # Infer the number of different classes from the labels of the support set
        n_way = len(torch.unique(support_labels))
        # Prototype i is the mean of all instances of features corresponding to labels == i
        z_proto = torch.cat(
            [
                z_support[torch.nonzero(support_labels == label)].mean(0)
                for label in range(n_way)
            ]
        )

    
        # Compute the euclidean distance from queries to prototypes
        dists = torch.cdist(z_query, z_proto)
   

        # And here is the super complicated operation to transform those distances into classification scores!
        scores = -dists
        
        return scores


In [30]:
convolutional_network = CNN()
model = PrototypicalNetworks(convolutional_network)
print(model)

PrototypicalNetworks(
  (backbone): CNN(
    (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (fc1): Linear(in_features=100352, out_features=256, bias=True)
    (fc2): Linear(in_features=256, out_features=10, bias=True)
  )
)


In [31]:
N_WAY = 4  # Number of classes in a task
N_SHOT = 4 # Number of images per class in the support set
N_QUERY = 2 # Number of images per class in the query set
N_EVALUATION_TASKS = 1

# The sampler needs a dataset with a "get_labels" method. Check the code if you have any doubt!
test_set.get_labels = lambda: [
    instance[1] for instance in test_set.img_labels
]

test_sampler = TaskSampler(
    test_set, n_way=N_WAY , n_shot=N_SHOT, n_query=N_QUERY, n_tasks=N_EVALUATION_TASKS
)

test_loader = DataLoader(
    test_set,
    batch_sampler=test_sampler,
    num_workers=0,
    pin_memory=True,
    collate_fn=test_sampler.episodic_collate_fn,
)

In [32]:
(
    example_support_images,
    example_support_labels,
    example_query_images,
    example_query_labels,
    example_class_ids,
) = next(iter(test_loader))

(512, 512)


100%|██████████| 3/3 [00:00<00:00, 3012.43it/s]


(512, 512)


100%|██████████| 2/2 [00:00<?, ?it/s]


(512, 512)


...

100%|██████████| 3/3 [00:00<?, ?it/s]


(512, 512)


100%|██████████| 2/2 [00:00<00:00, 2005.88it/s]


(512, 512)


100%|██████████| 2/2 [00:00<?, ?it/s]


In [33]:
model.eval()

print(type(example_support_images))
print(len(example_support_images))

print(type(example_support_images[0]))

example_scores = model(
    example_support_images,
    example_support_labels,
    example_query_images,
).detach()


_, example_predicted_labels = torch.max(example_scores.data, 1)

print(example_query_labels)
print(example_predicted_labels)

<class 'torch.Tensor'>
16
<class 'torch.Tensor'>
tensor([3, 3, 2, 2, 1, 1, 0, 0])
tensor([3, 0, 2, 2, 0, 0, 0, 3])


In [34]:
def evaluate_on_one_task(
    support_images: torch.Tensor,
    support_labels: torch.Tensor,
    query_images: torch.Tensor,
    query_labels: torch.Tensor,
) -> [int, int]:
    return (
         torch.max(  
            model(support_images, support_labels, query_images).detach().data,1,)[1]
         )


def evaluate(data_loader: DataLoader):

    model.eval()
    with torch.no_grad():
        for episode_index, (
            support_images,
            support_labels,
            query_images,
            query_labels,
            class_ids,
        ) in tqdm(enumerate(data_loader), total=len(data_loader)):

            predicted_labels =evaluate_on_one_task(support_images, support_labels, query_images, query_labels)
            actual_labels  = query_labels

            actual_labels_np = actual_labels.cpu().numpy()
            predicted_labels_np = predicted_labels.cpu().numpy()

  
            precision = precision_score(actual_labels_np, predicted_labels_np, average='macro')
            recall = recall_score(actual_labels, predicted_labels, average='macro')
            f1_score_macro = f1_score(actual_labels, predicted_labels, average='macro')
            
            # Calculate accuracy
            accuracy = accuracy_score(actual_labels, predicted_labels)

      
            print("Precision (Macro):", precision)
            print("Recall (Macro):", recall)
            print("F1 Score (Macro):", f1_score_macro)
            print("Accuracy:", accuracy)

            print(actual_labels_np)
            print(predicted_labels_np)

Training a meta-learning algorithm


In [35]:
N_TRAINING_EPISODES = 6


train_set.get_labels = lambda: [ instance[1] for instance in train_set.img_labels]

train_sampler = TaskSampler(
    train_set, n_way=N_WAY, n_shot=N_SHOT, n_query=N_QUERY, n_tasks=N_TRAINING_EPISODES
)
train_loader = DataLoader(
    train_set,
    batch_sampler=train_sampler,
    num_workers=0,
    pin_memory=True,
    collate_fn=train_sampler.episodic_collate_fn,
)

In [36]:
def sliding_average(lst, window_size):
    if window_size == 0:
        return 0.0
    return sum(lst[-window_size:]) / min(len(lst), window_size)

In [37]:
from tqdm import tqdm
import matplotlib.pyplot as plt

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def fit(
    support_images: torch.Tensor,
    support_labels: torch.Tensor,
    query_images: torch.Tensor,
    query_labels: torch.Tensor,
) -> float:
    optimizer.zero_grad()
    classification_scores = model(
        support_images, support_labels, query_images
    )

    loss = criterion(classification_scores, query_labels)
    loss.backward()
    optimizer.step()

    return loss.item()

In [38]:
log_update_frequency = 10

all_loss = []
model.train()
with tqdm(enumerate(train_loader), total=len(train_loader)) as tqdm_train:
    
    for episode_index, (
        support_images,
        support_labels,
        query_images,
        query_labels,
        _,
    ) in tqdm_train:
        
        loss_value = fit(support_images, support_labels, query_images, query_labels)
        
        all_loss.append(loss_value)
      
        if episode_index % log_update_frequency == 0:
            tqdm_train.set_postfix(loss=sliding_average(all_loss, log_update_frequency))

  0%|          | 0/6 [00:00<?, ?it/s]

(512, 512)


100%|██████████| 3/3 [00:00<00:00, 3005.95it/s]


(512, 512)


100%|██████████| 2/2 [00:00<00:00, 2004.93it/s]


...

100%|██████████| 2/2 [00:00<00:00, 1331.95it/s]


(512, 512)


100%|██████████| 10/10 [00:00<00:00, 10034.22it/s]


(512, 512)


100%|██████████| 6/6 [01:16<00:00, 12.81s/it, loss=3.29]


In [39]:
evaluate(test_loader)

  0%|          | 0/1 [00:00<?, ?it/s]

(512, 512)


100%|██████████| 2/2 [00:00<?, ?it/s]


(512, 512)


100%|██████████| 2/2 [00:00<?, ?it/s]


...

(512, 512)


100%|██████████| 3/3 [00:00<?, ?it/s]


(512, 512)


100%|██████████| 1/1 [00:11<00:00, 11.74s/it]

Precision (Macro): 0.9166666666666666
Recall (Macro): 0.875
F1 Score (Macro): 0.8666666666666666
Accuracy: 0.875
[0 0 2 2 3 3 1 1]
[0 0 2 2 1 3 1 1]


In [40]:
torch.save(model.state_dict(), 'model.pth')